<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GROK4_XAI_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## XAIKEY

In [ ]:
!pip install xai-sdk -q

In [2]:
from xai_sdk import Client
from xai_sdk.chat import user, system

from google.colab import userdata
XAI_key = userdata.get('XAI_KEY')

# 1. Initialize the client
client = Client(
    api_host="api.x.ai",
    api_key=XAI_key # You need to replace this placeholder with your actual API key.
)

# 2. Create a chat session with the Grok 4 model (grok-4-0709 refers to Grok 4 released on July 9, 2025)
chat = client.chat.create(model="grok-4-0709", temperature=0)

# 3. Append system and user messages to the chat history
chat.append(system("You are a PhD-level mathematician."))
chat.append(user("Consider a $2025 \\times 2025$ grid of integers. Prove or disprove the following statement: If the sum of the integers in every $2 \\times 2$ subgrid is divisible by 4, then the sum of the integers in the entire $2025 \\times 2025$ grid is also divisible by 4. Provide a rigorous mathematical argument using techniques from linear algebra or modular arithmetic to support your conclusion."))

# 4. Sample (generate) a response from the model
response = chat.sample()

# 5. Print the generated response content
print(response.content)

### Disproof of the Statement

The statement is false. We will disprove it by constructing an explicit counterexample where every $2 \times 2$ subgrid sum is divisible by 4, but the total sum of the $2025 \times 2025$ grid is not divisible by 4. The construction relies on the linear structure of the conditions modulo 4 and exploits the odd dimension of the grid.

#### Structure of Grids Satisfying the $2 \times 2$ Conditions

Denote the grid entries by $a_{i,j} \in \mathbb{Z}$ for $1 \leq i,j \leq 2025$. The condition is that for all $1 \leq k,\ell \leq 2024$,
\[
a_{k,\ell} + a_{k,\ell+1} + a_{k+1,\ell} + a_{k+1,\ell+1} \equiv 0 \pmod{4}.
\]

These conditions relate consecutive rows. Specifically, for fixed rows $i$ and $i+1$ (with $1 \leq i \leq 2024$), the entries in row $i+1$ satisfy, for each $1 \leq j \leq 2024$,
\[
a_{i+1,j+1} \equiv -a_{i,j} - a_{i,j+1} - a_{i+1,j} \pmod{4}.
\]

This allows a recursive construction: 
- Choose all entries in row 1 arbitrarily (in $\mathbb{Z}$).
-

## Using XAI SDK with Langchain

Since using the XAI model directly with CrewAI via LiteLLM is encountering compatibility issues, we can use the `xai_sdk` within the Langchain framework. This involves creating a custom Langchain `BaseLLM` (or `LLM`) wrapper that uses the `xai_sdk.Client` to interact with the Grok-4 model.

In [ ]:
!pip install langchain_core -q
!pip install langchain -q
!pip install xai-sdk -q
!pip install langgraph -q

In [1]:
import os
from typing import Any, List, Mapping, Optional, Dict, TypedDict, Annotated, Union
import operator
import json

# --- Correcting Imports ---
from langchain_core.language_models.llms import LLM
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.tools import tool
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
# Corrected import path for AgentOutputParser
from langchain.agents.agent import AgentOutputParser
from langchain_core.runnables import RunnablePassthrough
from langgraph.graph import StateGraph, END
from langchain.schema.agent import AgentAction, AgentFinish

from xai_sdk import Client
from xai_sdk.chat import user, system

# --- Configuration ---
# NOTE: Ensure 'XAI_KEY' is set in your Colab secrets
try:
    from google.colab import userdata
    XAI_API_KEY = userdata.get('XAI_KEY')
except ImportError:
    print("WARNING: Not running in Google Colab. Please ensure XAI_KEY is set manually if running elsewhere.")
    XAI_API_KEY = os.environ.get('XAI_KEY')

if XAI_API_KEY is None:
    print("WARNING: Please set your XAI_KEY in Colab secrets or environment variables.")

# --- Custom Grok 4 Langchain LLM Wrapper ---
class Grok4LangchainLLM(LLM):
    """
    Custom Langchain LLM wrapper for xAI's Grok 4 model.
    """
    client: Client
    model_name: str = "grok-4-0709"
    temperature: float = 0.7

    def __init__(self, api_key: str, model_name: str = "grok-4-0709", temperature: float = 0.7, **kwargs):
        if api_key is None:
            raise ValueError("API key must be provided for Grok4LangchainLLM.")

        xai_client_instance = Client(api_key=api_key)

        super().__init__(
            client=xai_client_instance,
            model_name=model_name,
            temperature=temperature,
            **kwargs
        )

    @property
    def _llm_type(self) -> str:
        return "grok4_xai"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        try:
            chat = self.client.chat.create(model=self.model_name, temperature=self.temperature)
            chat.append(user(prompt))

            response = chat.sample()
            return response.content
        except Exception as e:
            # We must return a string even in error cases for the LLM interface
            print(f"Error calling Grok 4 API via Langchain wrapper: {e}")
            return f"Error: Could not get a response from Grok 4. {e}"

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {
            "model_name": self.model_name,
            "temperature": self.temperature,
            "api_key": "..."
            }

# --- Instantiate the Custom LLM ---
grok4_langchain_llm = None
if XAI_API_KEY:
    try:
        grok4_langchain_llm = Grok4LangchainLLM(api_key=XAI_API_KEY)
        print(f"Custom Grok 4 Langchain LLM instantiated with model: {grok4_langchain_llm.model_name}")
    except Exception as e:
        print(f"Failed to instantiate Grok 4 LLM: {e}")
else:
    print("XAI API Key not found. Cannot instantiate custom LLM.")

# --- 1. Define Tools ---
@tool
def flight_search_tool(origin: str, destination: str, date: str) -> List[Dict[str, Any]]:
    """Searches for flight options between an origin and a destination on a specific date.
    Args:
        origin (str): The departure location (e.g., London).
        destination (str): The arrival location (e.g., Tokyo).
        date (str): The date of the flight in YYYY-MM-DD format (e.g., 2025-08-15).
    """
    print(f"\n--- Tool Executing: flight_search_tool ({origin} -> {destination} on {date}) ---")

    # Simulate API response
    if origin == "London" and destination == "Tokyo" and date == "2025-08-15":
        return [
            {"flight_number": "BA007", "airline": "British Airways", "price": 1200, "duration_hours": 13, "layovers": 0},
            {"flight_number": "JAL404", "airline": "Japan Airlines", "price": 1150, "duration_hours": 14, "layovers": 1}
        ]
    else:
        return {"message": "No flights found or invalid input."}

tools = [flight_search_tool]

# --- 2. Setup Grok 4-Powered Agents ---
if grok4_langchain_llm:
    # Custom output parser for Grok 4
    class Grok4ToolOutputParser(AgentOutputParser):
        def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
            # Attempt to parse Grok 4's output assuming it followed the JSON instruction for tool calling

            try:
                # Grok 4's output will be a raw string. We try to interpret it as JSON.
                response_data = json.loads(llm_output)

                # Check for the 'tool_calls' key indicating a tool action
                if 'tool_calls' in response_data and isinstance(response_data['tool_calls'], list) and response_data['tool_calls']:
                    # Extract the first tool call
                    tool_call = response_data['tool_calls'][0]

                    if 'name' in tool_call and 'args' in tool_call:
                        # Return as an AgentAction
                        return AgentAction(
                            tool=tool_call['name'],
                            tool_input=tool_call['args'],
                            log=llm_output
                        )

                # Check for 'output' key indicating a final answer
                if 'output' in response_data:
                    return AgentFinish(
                        return_values={'output': response_data['output']},
                        log=llm_output
                    )

            except (json.JSONDecodeError, KeyError, IndexError):
                # If JSON parsing fails or keys are missing, treat the output as a final answer
                pass

            # Default behavior: If not a structured tool call or final answer, return as a final answer
            return AgentFinish(
                return_values={"output": llm_output},
                log=llm_output
            )

    # Define the system prompt with clear JSON output instructions for tool use
    tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])
    system_prompt_text = (
        f"You are a professional flight search assistant. You have access to the following tools:\n\n{tool_descriptions}\n\n"
        "You must respond using a JSON object containing either 'tool_calls' if you need to execute a tool, or 'output' if you have a final answer. "
        "When calling a tool, use the format: {{ \"tool_calls\": [{{ \"name\": \"tool_name\", \"args\": {{ \"arg1\": \"value1\", ... }} }}] }}. "
        "When providing a final answer, use the format: {{ \"output\": \"Your final answer here.\" }}. "
        "Ensure your response is valid JSON. If you cannot fulfill the request with the tools, provide a final answer."
    )

    # Agent 1: Flight Search Agent Chain
    flight_search_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt_text),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )

    # We chain the prompt, LLM, and the custom output parser
    flight_search_agent_chain = (
        RunnablePassthrough.assign(
            # We format intermediate steps (previous tool calls and observations)
            agent_scratchpad=lambda x: format_to_openai_tool_messages(x["intermediate_steps"])
        )
        | flight_search_prompt
        | grok4_langchain_llm
        | Grok4ToolOutputParser() # Use our custom parser
    )

    # Agent Executor for the Flight Search Agent
    flight_search_executor = AgentExecutor(
        agent=flight_search_agent_chain,
        tools=tools,
        verbose=True,
        # We rely on the AgentExecutor to handle the loop based on the AgentAction/AgentFinish from our custom parser
        handle_parsing_errors=True
    )

    # Agent 2: Review Agent (A simple chain for summarizing)
    review_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a flight review specialist. Analyze the provided flight search results and summarize them concisely for the user. Highlight the cheapest option."),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    review_chain = review_prompt | grok4_langchain_llm

    # --- 3. Orchestration with LangGraph ---

    class AgentState(TypedDict):
        messages: Annotated[List[Any], operator.add]

    def execute_flight_search(state):
        print("\n--- Executing Flight Search Agent (Grok 4) ---")

        # Invoke the Agent Executor.
        result = flight_search_executor.invoke({"messages": state["messages"]})

        # Return the output of the AgentExecutor.
        return {"messages": [AIMessage(content=result.get("output", "No output from agent."))]}

    def review_results(state):
        print("\n--- Executing Review Agent (Grok 4) ---")

        # The Review Agent processes the full message history
        review_output = review_chain.invoke({"messages": state["messages"]})

        # Return the summarized results
        return {"messages": [AIMessage(content=review_output)]}

    # Build the LangGraph workflow
    workflow = StateGraph(AgentState)
    workflow.add_node("flight_search", execute_flight_search)
    workflow.add_node("review_results", review_results)

    # Define the edges (flow: Search -> Review -> END)
    workflow.set_entry_point("flight_search")
    workflow.add_edge("flight_search", "review_results")
    workflow.add_edge("review_results", END)

    # Compile the graph
    app = workflow.compile()
    print("LangGraph workflow compiled using Grok 4 agents.")

    # --- 4. Running the Demo ---
    user_query = "Find me flights from London to Tokyo for August 15, 2025."
    print(f"\n--- Starting Multi-Agent Flight Planning Demo (Grok 4.0 ) ---")
    print(f"User Query: {user_query}")

    # Run the graph
    inputs = {"messages": [HumanMessage(content=user_query)]}

    try:
        final_output = app.invoke(inputs)

        print("\n--- Final Multi-Agent Output ---")
        print(final_output["messages"][-1].content)

    except Exception as e:
        print(f"\nError during LangGraph execution: {e}")
        print("Please verify the XAI API Key and ensure the Grok 4 service is accessible.")

else:
    print("\nCannot run multi-agent demo without a valid Grok 4 LLM instantiation.")

Custom Grok 4 Langchain LLM instantiated with model: grok-4-0709
LangGraph workflow compiled using Grok 4 agents.

--- Starting Multi-Agent Flight Planning Demo (Grok 4.0 ) ---
User Query: Find me flights from London to Tokyo for August 15, 2025.

--- Executing Flight Search Agent (Grok 4) ---


> Entering new AgentExecutor chain...
{"tool_calls":[{"name":"flight_search_tool","args":{"origin":"London","destination":"Tokyo","date":"2025-08-15"}}]}
--- Tool Executing: flight_search_tool (London -> Tokyo on 2025-08-15) ---
[{'flight_number': 'BA007', 'airline': 'British Airways', 'price': 1200, 'duration_hours': 13, 'layovers': 0}, {'flight_number': 'JAL404', 'airline': 'Japan Airlines', 'price': 1150, 'duration_hours': 14, 'layovers': 1}]{"output": "Based on the search, here are some available flights from London to Tokyo on August 15, 2025:\n\n1. British Airways - Departure: 09:00 from LHR, Arrival: 06:30+1 at NRT, Duration: 11h 30m, Price: $1200 (Economy).\n2. Japan Airlines - Departur